In [1]:
from __future__ import division

import math
import joblib
import os.path
import glob
import pretty_midi
import numpy as np

import argparse
import librosa
import os
import sys

In [2]:
workdir = "C:\\Users\\toend\\Documents\\ITU\\Thesis"
path = os.path.join(workdir, "MIDIdata\\2002")
pathToSave = os.path.join(workdir, "MIDIdata\\cuts\\02cuts")
sequenceLength = 30 #seconds
frequency = 100 

In [3]:
#I believe this will only cut like this [0, 1, 2, 3, 4], [5, 6, 7, 8, 9] --> [0, 1, 2], [3, 4, 5], [6, 7, 8], [9]
#def reshape(piano_roll):
#    return np.reshape(piano_roll, (len(piano_roll[0]), 128))

In [4]:
def reshape(piano_roll):
    h, w = piano_roll.shape
    slices = []
    for i in range(w):
        columnSlice = piano_roll[:,i]
        columnSlice = np.asarray(columnSlice)
        slices.append(columnSlice)
    return np.asarray(slices)

In [5]:
def make0or1(piano_roll):
    #Need to convert all "volume" values (f.ex. 47) to just values of 1 to not confuse the network
    simplified_piano_roll = np.where(piano_roll==0, piano_roll, 80)
    return simplified_piano_roll

In [6]:
def trimPianoRoll(piano_roll):
    height, width = piano_roll.shape
    tail = width % (sequenceLength*frequency)
    if tail == 0:
        return piano_roll
    else: 
        trimmedPianoRoll = piano_roll[:, :-tail]
        return trimmedPianoRoll

In [7]:
def padPianoRoll(piano_roll):
    height, width = piano_roll.shape
    padWidth = (sequenceLength*frequency) - (width % (sequenceLength*frequency))
    padding = np.zeros((height, padWidth))
    paddedPianoRoll = np.append(piano_roll, padding, axis=1)
    return paddedPianoRoll

In [8]:
def slicePianoRoll(piano_roll):
    #Need to pad or trim to avoid splitting error
    #paddedPianoRoll = padPianoRoll(piano_roll)
    trimmedPianoRoll = trimPianoRoll(piano_roll)
    #Cutting based on length in seconds * frequency
    height, width = trimmedPianoRoll.shape
    numberOfSlices = width/(sequenceLength*frequency)
    #Need to slice the piano_roll vertically to get an array based on timesteps instead of an array based on nodes
    return np.hsplit(trimmedPianoRoll, numberOfSlices)    
    

In [9]:
def computePianoRoll(midi_file):
    try:
        midi = pretty_midi.PrettyMIDI(midi_file)
        #Retrieving only the piano track
        piano_midi = midi.instruments[0]
        #piano_midi = midi
        #Splitting each track up into (1/frequency) second "windows"
        piano_roll = piano_midi.get_piano_roll(fs=frequency)
        return piano_roll
    except Exception as e:
        print(e)

In [10]:
def sliceMidi(midi_file):
    piano_roll = computePianoRoll(midi_file)
    slicedPianoRolls = slicePianoRoll(piano_roll)
    return slicedPianoRolls

In [11]:
def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [12]:
#testPath = os.path.join(workdir, "MIDIdata\\test")
#with os.scandir(testPath) as entries:
#    for entry in entries:
#        if entry.is_file():
#            filename = "02" + entry.name.replace(".mid", "")
#            midi_file = entry.path
#            saveTo = os.path.join(pathToSave, filename)
#            slicedPianoRolls = sliceMidi(midi_file)
#            for i in range(0, len(slicedPianoRolls)):
#                cut = slicedPianoRolls[i]
#                pm = piano_roll_to_pretty_midi(cut, frequency, 0)
#                cutName = filename + "-" + str(i*30)
#                pm.write(pathToSave + "\\" + cutName + ".mid")

In [13]:
#testPath = os.path.join(workdir, "MIDIdata\\test")
#with os.scandir(testPath) as entries:
#    for entry in entries:
#        if entry.is_file():
#            filename = "02" + entry.name.replace(".mid", "")
#            midi_file = entry.path
#            slicedPianoRolls = sliceMidi(midi_file)
#            cut = slicedPianoRolls[0]
#            piece = cut[59][1285]
#           print(piece)

In [14]:
#testPath = os.path.join(workdir, "MIDIdata\\test")
#with os.scandir(testPath) as entries:
#    for entry in entries:
#        if entry.is_file():
#            filename = "02" + entry.name.replace(".mid", "")
#            midi_file = entry.path
#            pm = pretty_midi.PrettyMIDI(midi_file)
#            notes = pm.instruments[0].notes
#            print(notes)
            
        
            #print(pm.instruments[0].control_changes)
            
            #PIANO ROLL IS LOSSY
            #piano_roll = pm.get_piano_roll()
            #piano_roll = computePianoRoll(midi_file)
            #reshaped = reshape(piano_roll)
            #h, w = reshaped.shape
            #print(reshaped[h-1])
            #for i in range(0, h):
            #    print(reshaped[i])
            #pm = piano_roll_to_pretty_midi(piano_roll, frequency, 0)
            #pm.write(pathToSave + "\\" + filename + "2.mid")

In [15]:
def splitNotes(notes):
    lengthOfTrack = notes[-1].end
    cuts = int(lengthOfTrack/sequenceLength)
    for i in range(0, cuts):
        print(i*sequenceLength)

In [16]:
def createPianoRoll(piano):
    endTime = math.ceil(piano.get_end_time())
    #print(endTime)
    notes = piano.notes
    #adding a +1 (column) to avoid index out of bounds
    width = (endTime*frequency)+1
    piano_roll = np.zeros((128, width))
    for note in notes:
        pitch = note.pitch
        velocity = note.velocity
        start = int(round(note.start * frequency))
        end = int(round(note.end * frequency))
        for i in range(start, end+1):
            piano_roll[pitch][i] = velocity
    return piano_roll

In [18]:
#testPath = os.path.join(workdir, "MIDIdata\\test")
with os.scandir(path) as entries:
    for entry in entries:
        if entry.is_file():
            if not ".ini" in entry.name:
                filename = "02" + entry.name.replace(".mid", "")
                filename = filename.replace(".MID", "")
                print(filename)
                midi_file = entry.path
                pm = pretty_midi.PrettyMIDI(midi_file)
                piano = pm.instruments[0]
                piano_roll = createPianoRoll(piano)
                cuts = slicePianoRoll(piano_roll)
                for i in range(0, len(cuts)):
                    cut = cuts[i]
                    #print(cut.shape)
                    pm = piano_roll_to_pretty_midi(cut, frequency, 0)
                    cutName = filename + "-" + str(i*30)
                    pm.write(pathToSave + "\\" + cutName + ".mid")

02chan01
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02chan02
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02cho01
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(1

(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02giltburg03
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02giltburg04
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02giltburg05
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(

(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02kolessova03
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02kolessova04
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02kolessova05
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000

(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02mamriev04
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02mamriev05
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02mikhailoff01
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02mikhailoff0

(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02soloman03
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128

(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02yamaguchi03
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
02yamaguchi04
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
(128, 3000)
